In [1]:
import metrica_io as io
import metrica_viz as viz
import metrica_velocity as mvelocity
import metrica_pitch_control as mpc

import matplotlib.pyplot as plt
import os

In [2]:
data_dir = 'D:\Football\Data\MetricaSports\data'
game_id = 2

In [3]:
events = io.read_event_data(data_dir,game_id)

tracking_home = io.read_tracking_data(data_dir,game_id,'Home')
tracking_away = io.read_tracking_data(data_dir,game_id,'Away')

# Convert positions from metrica units to meters (note change in Metrica's coordinate system since the last lesson)
tracking_home = io.convert_data(tracking_home)
tracking_away = io.convert_data(tracking_away)
events = io.convert_data(events)

Reading team: home
Reading team: away


In [10]:
shots = events[ events['Type'] == 'SHOT']
shots

,Team,Type,Subtype,Period,Start Frame,Start Time [s],End Frame,End Time [s],From,To,Start X,Start Y,End X,End Y
83,Home,SHOT,OFF TARGET-OUT,1,4419,176.76,4443,177.72,Player5,NaN,-38.85,18.36,-54.60,2.72
198,Home,SHOT,ON TARGET-GOAL,1,12202,488.08,12212,488.48,Player10,NaN,-47.25,-3.40,-53.55,-2.04
252,Home,SHOT,ON TARGET-SAVED,1,16484,659.36,16499,659.96,Player10,NaN,-40.95,11.56,-48.30,0.68
297,Away,SHOT,BLOCKED,1,18515,740.60,18520,740.80,Player22,NaN,31.50,10.20,37.80,11.56
431,Home,SHOT,HEAD-OFF TARGET-OUT,1,27345,1093.80,27360,1094.40,Player3,NaN,-43.05,-6.12,-53.55,-6.12
472,Home,SHOT,ON TARGET-SAVED,1,29754,1190.16,29777,1191.08,Player4,NaN,-31.50,-13.60,-52.50,-2.72
823,Away,SHOT,ON TARGET-GOAL,1,53049,2121.96,53075,2123.00,Player24,NaN,44.10,2.72,53.55,1.36
852,Home,SHOT,HEAD-OFF TARGET-OUT,1,56079,2243.16,56131,2245.24,Player5,NaN,-42.00,6.12,-54.60,-17.68
962,Away,SHOT,OFF TARGET-HEAD-OUT,1,63362,2534.48,63372,2534.88,Player23,NaN,45.15,0.68,53.55,-6.80
980,Away,SHOT,ON TARGET-SAVED,1,64772,2590.88,64799,2591.96,Player20,NaN,28.35,-3.40,51.45,-2.04


In [4]:
# Calculate player velocities
tracking_home = mvelocity.calc_player_velocities(tracking_home,smoothing=True)
tracking_away = mvelocity.calc_player_velocities(tracking_away,smoothing=True)

In [5]:
tracking_home, tracking_away, events = io.to_single_half(tracking_home, tracking_away, events)

In [13]:
import moviepy.editor as mpy
from moviepy.video.io.bindings import mplfig_to_npimage
import math


starting_frame = 89287
end_frame = 90180
f = 25

In [1]:
def make_frame(t):
    t2 = int(math.ceil(t*f+0.0001)-1)
    #print(t ,t2+starting_frame, 'dscsd')
    
    fig, ax = viz.plot_Delaunay_for_frame(t2+starting_frame, tracking_home, tracking_away, team='Away', include_player_velocities=True )

    image = mplfig_to_npimage(fig)
    return image # returns a 8-bit RGB array

clip = mpy.VideoClip(make_frame, duration=((end_frame-starting_frame)/f)).set_fps(f) # 2 seconds
clip.write_videofile("sample-away.mp4")#video file is saved

NameError: name 'mpy' is not defined